<a href="https://colab.research.google.com/github/oxanaRC/CE902/blob/main/Constructing_Historic_Median_Sales_Price_per_Postal_Area_and_per_bedroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

14.07.21 - 19.07.21
Identifying PostCode areas in the property sales dataset (10 months - Jan, Feb, Mar, Apr, May, June, Jul, Aug, Sep, Oct 2018) and obtaining Median Sales Price per bedroom per PostCode area.



In [1]:
!pip install harperdb

In [2]:
import harperdb
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
url='https://ce901data-orc0770.harperdbcloud.com'
db=harperdb.HarperDB(url=url,
            username='myuser',
            password='abcdef',
            timeout=500)

In [4]:
db

Below data is extracted from the SQL database, however because of poor performance of the JOIN SQL operations, instead of relational SQL statements, the JOIN operations are performed in pandas DataFrames (according to https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).


The data extraction from SQL tables in placed within the loop to avoid the database timeout issues.


In [5]:
list_of_dfs = {}
count=0
for mt in range(1, 11):
  for yr in range (2018,2019):
    year=yr
    month=mt
    count=count+1
    #sqlstring1='SELECT bedrooms.id, bedrooms.num_of_beds, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date '
    #sqlstring2='FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE (num_of_beds>0 and deed_date LIKE "%{}/{}")'
    #sqlstring2='FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE deed_date LIKE "%{}/{}"'
    #sqlstring=sqlstring1+sqlstring2
    sqlstring='SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%{}/{}"'
    sqlstring=sqlstring.format(month,year)
    print(sqlstring)
    my_db_data=db.sql(sqlstring)
    list_of_dfs[count] = pd.DataFrame(my_db_data)
    print(list_of_dfs[count].shape)
df_prop_sales = pd.concat(list_of_dfs, sort=False)
#excluding 'other' property types (i.e. commercial) from the properties sales data 
df_prop_sales=df_prop_sales[(df_prop_sales['property_type']!='O')]
df_prop_sales

SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%1/2018"
(169065, 8)
SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%2/2018"
(157760, 8)
SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%3/2018"
(84147, 8)
SELECT propertypurchases.unique_id, propertypurchases

unique_id  ...  postcode
1  0      64342BFD-ABA7-422C-E053-6C04A8C0FB8A  ...    E8 4NE
   1      64342BFD-ABAA-422C-E053-6C04A8C0FB8A  ...    N5 1BZ
   2      64342BFD-ABAB-422C-E053-6C04A8C0FB8A  ...   W11 1NZ
   3      64342BFD-ABAC-422C-E053-6C04A8C0FB8A  ...   IG3 9NB
   4      64342BFD-ABB0-422C-E053-6C04A8C0FB8A  ...   W12 8JN
...                                        ...  ...       ...
10 93062  C18F412B-1446-81A6-E053-6B04A8C0AD18  ...  CH42 2AQ
   93063  C18F412B-14DD-81A6-E053-6B04A8C0AD18  ...   L36 9TG
   93064  C18F412B-795D-81A6-E053-6B04A8C0AD18  ...   B31 2NA
   93065  C18F412B-94F2-81A6-E053-6B04A8C0AD18  ...   LS7 4AG
   93066  C18F412B-97A6-81A6-E053-6B04A8C0AD18  ...   E14 9XD

[964028 rows x 8 columns]

In [6]:
sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds FROM projectdata.bedrooms WHERE num_of_beds>0'
bedrooms=db.sql(sqlstring)
df_bedrooms=pd.DataFrame(bedrooms)
df_bedrooms.rename(columns = {'id':'unique_id'}, inplace = True)
df_bedrooms

,unique_id,num_of_beds
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1
1,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1
2,79A74E21-DB3C-1289-E053-6B04A8C01627,1
3,79A74E21-1F11-1289-E053-6B04A8C01627,1
4,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1
...,...,...
550498,773788C2-A90F-2CE4-E053-6C04A8C05E57,17
550499,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17
550500,64342BFF-158B-422C-E053-6C04A8C0FB8A,23
550501,75050A86-10FB-9A88-E053-6B04A8C02390,46


In [7]:
#sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE num_of_beds>0'
#my_db_data=db.sql(sqlstring)
#data=pd.DataFrame(my_db_data)
data = pd.merge(df_bedrooms, df_prop_sales, how="left", on="unique_id")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
2,79A74E21-DB3C-1289-E053-6B04A8C01627,1,33000.0,LIVERPOOL,LIVERPOOL,L,F,21/09/2018,L5 4TD
3,79A74E21-1F11-1289-E053-6B04A8C01627,1,405000.0,LONDON,HAMMERSMITH AND FULHAM,L,F,27/09/2018,W14 9BW
4,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH
...,...,...,...,...,...,...,...,...,...
550498,773788C2-A90F-2CE4-E053-6C04A8C05E57,17,1011992.0,LYNDHURST,NEW FOREST,F,D,10/08/2018,SO43 7BT
550499,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA
550500,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA
550501,75050A86-10FB-9A88-E053-6B04A8C02390,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#check rows where postcode is NAN - these are the result of rows added on the Join with bedrooms data where bedrooms data is for prop sale which happened after the considered period
#data_t=data[ (data['postcode'].isnull()) ]
#drop NAN rows from data
data=data.dropna(subset=['postcode'])     
data 

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
2,79A74E21-DB3C-1289-E053-6B04A8C01627,1,33000.0,LIVERPOOL,LIVERPOOL,L,F,21/09/2018,L5 4TD
3,79A74E21-1F11-1289-E053-6B04A8C01627,1,405000.0,LONDON,HAMMERSMITH AND FULHAM,L,F,27/09/2018,W14 9BW
4,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH
...,...,...,...,...,...,...,...,...,...
550497,773788C2-69BA-2CE4-E053-6C04A8C05E57,16,1445000.0,AXMINSTER,EAST DEVON,F,D,25/07/2018,EX13 8AA
550498,773788C2-A90F-2CE4-E053-6C04A8C05E57,17,1011992.0,LYNDHURST,NEW FOREST,F,D,10/08/2018,SO43 7BT
550499,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA
550500,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA


In [10]:
#checking duplicates (some bedroom data was loaded twice by mistake)
duplicate = data[data.duplicated()]
duplicate

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
204,79A74E22-514F-1289-E053-6B04A8C01627,1,155000.0,WARWICK,WARWICK,L,F,03/10/2018,CV34 4NL
754,79A74E21-9309-1289-E053-6B04A8C01627,1,185000.0,WELWYN GARDEN CITY,WELWYN HATFIELD,L,F,28/09/2018,AL7 3XA
1745,6B32222C-B011-01F1-E053-6C04A8C0D2C8,1,97000.0,SHEFFIELD,SHEFFIELD,L,F,21/02/2018,S13 7BW
2826,68FEB20C-2AC0-38DA-E053-6C04A8C051AE,1,173500.0,EAST GRINSTEAD,MID SUSSEX,L,F,09/02/2018,RH19 1BT
2906,7C2D0700-D684-4963-E053-6B04A8C07B97,1,120000.0,BIRMINGHAM,BIRMINGHAM,L,F,28/09/2018,B17 0JJ
...,...,...,...,...,...,...,...,...,...
550011,80E1AA98-CCBA-7BF8-E053-6C04A8C00BF2,7,386000.0,WARWICK,WARWICK,F,T,14/09/2018,CV35 8BU
550036,666758D7-5352-3363-E053-6B04A8C0D74E,7,795000.0,SHEFFIELD,ROTHERHAM,F,D,14/02/2018,S26 3YD
550240,7C2D0701-2821-4963-E053-6B04A8C07B97,8,331000.0,BIRMINGHAM,BIRMINGHAM,F,T,26/10/2018,B29 7DL
550359,773788C3-5811-2CE4-E053-6C04A8C05E57,9,217000.0,COVENTRY,COVENTRY,F,T,07/09/2018,CV4 8FR


In [11]:
data=data.drop_duplicates()
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
2,79A74E21-DB3C-1289-E053-6B04A8C01627,1,33000.0,LIVERPOOL,LIVERPOOL,L,F,21/09/2018,L5 4TD
3,79A74E21-1F11-1289-E053-6B04A8C01627,1,405000.0,LONDON,HAMMERSMITH AND FULHAM,L,F,27/09/2018,W14 9BW
4,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH
...,...,...,...,...,...,...,...,...,...
550497,773788C2-69BA-2CE4-E053-6C04A8C05E57,16,1445000.0,AXMINSTER,EAST DEVON,F,D,25/07/2018,EX13 8AA
550498,773788C2-A90F-2CE4-E053-6C04A8C05E57,17,1011992.0,LYNDHURST,NEW FOREST,F,D,10/08/2018,SO43 7BT
550499,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA
550500,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA


In [12]:
#data preparation: 1. new column: postcode area with month and year and (eg. NR3_04_2018) 2. new column: calculating price per bedroom (price paid/number of bedrooms)
data_prep=data
data_prep['postcode_area_m_y']=data_prep['postcode'].str.split(' ').str[0]+'_'+data_prep['deed_date'].str.split('/').str[1]+'_'+data_prep['deed_date'].str.split('/').str[2]
data_prep['price_per_bedroom']=data_prep['price_paid']/data_prep['num_of_beds']
data_prep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,M30_07_2018,100000.000000
1,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,NR3_04_2018,128000.000000
2,79A74E21-DB3C-1289-E053-6B04A8C01627,1,33000.0,LIVERPOOL,LIVERPOOL,L,F,21/09/2018,L5 4TD,L5_09_2018,33000.000000
3,79A74E21-1F11-1289-E053-6B04A8C01627,1,405000.0,LONDON,HAMMERSMITH AND FULHAM,L,F,27/09/2018,W14 9BW,W14_09_2018,405000.000000
4,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH,CB9_03_2018,156500.000000
...,...,...,...,...,...,...,...,...,...,...,...
550497,773788C2-69BA-2CE4-E053-6C04A8C05E57,16,1445000.0,AXMINSTER,EAST DEVON,F,D,25/07/2018,EX13 8AA,EX13_07_2018,90312.500000
550498,773788C2-A90F-2CE4-E053-6C04A8C05E57,17,1011992.0,LYNDHURST,NEW FOREST,F,D,10/08/2018,SO43 7BT,SO43_08_2018,59528.941176
550499,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA,NP4_01_2018,8235.294118
550500,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA,WV2_01_2018,3478.260870


In [32]:
#for testing calculations for a specific area
df_co4=data_prep[(data_prep['postcode_area_m_y']=='CO4_08_2018')]
co4_median_price_per_bed = df_co4.groupby(['postcode_area_m_y'])['price_per_bedroom'].median().reset_index()
co4_median_price_per_bed

,postcode_area_m_y,price_per_bedroom
0,CO4_08_2018,94500.0


In [13]:
#function which will be used to work as follows -> e.g. we have 'postcode_area_m_y' M30_07_2018 - passing this as parameter to the setThreeMonthsPlus(M30,7,2018) - function returns 'M30_10_2018)
def setThreeMonthsPlus(anId,aMonth,aYear):
  if aMonth>=10:
    tMonth=aMonth+3-12
    tYear=aYear+1
  else:
    tMonth=aMonth+3
    tYear=aYear
  myNewId= str(anId).split('_')[0]+'_'+str(tMonth).zfill(2) +'_'+str(tYear)
  return myNewId

In [14]:
#creating new dataframe which stores median price_per_bedroom for each postcode_area_m_y
g_median_price_per_bed = data_prep.groupby(['postcode_area_m_y'])['price_per_bedroom'].median().reset_index()
g_median_price_per_bed['m_y']=g_median_price_per_bed['postcode_area_m_y'].str[-7:]
g_median_price_per_bed

,postcode_area_m_y,price_per_bedroom,m_y
0,AL10_01_2018,117916.666667,01_2018
1,AL10_02_2018,121666.666667,02_2018
2,AL10_03_2018,118666.666667,03_2018
3,AL10_04_2018,131666.666667,04_2018
4,AL10_05_2018,122500.000000,05_2018
...,...,...,...
23890,YO8_06_2018,60000.000000,06_2018
23891,YO8_07_2018,66000.000000,07_2018
23892,YO8_08_2018,62083.333333,08_2018
23893,YO8_09_2018,65500.000000,09_2018


In [15]:
#Need to add the following assumption: if data belongs to 1/2018, 2/2018 and 3/2018 - we do not have historic correspondong data for 10/2017, 11/2017 and 12/2017,
#so the working assumption is that median prices per bedroom in 10/2017 were the same as in 1/2018, median prices per bedroom in 11/2017 were the same as in 2/2018, median prices per bedroom in 12/2017 were the same as in 3/2018.
#To implement this assumption - we'll be getting the section of the above table for  1/2018, 2/2018 and 3/2018 so this will be appended to the new table as price_per_bedroom_3mnths_ago
df_add_assumption=g_median_price_per_bed[(g_median_price_per_bed['m_y']=='01_2018')|(g_median_price_per_bed['m_y']=='02_2018')|(g_median_price_per_bed['m_y']=='03_2018')]
df_add_assumption.rename(columns = {'price_per_bedroom':'median_ppb_3mnths_ago'}, inplace = True)
df_add_assumption=df_add_assumption.drop(['m_y'], axis = 1)
df_add_assumption

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_01_2018,117916.666667
1,AL10_02_2018,121666.666667
2,AL10_03_2018,118666.666667
12,AL1_01_2018,189083.333333
13,AL1_02_2018,183202.500000
...,...,...
23876,YO7_02_2018,67333.333333
23877,YO7_03_2018,55000.000000
23885,YO8_01_2018,63500.000000
23886,YO8_02_2018,61000.000000


In [17]:
# looping through each row in the g_median_price_per_bed and obtaining 'three months plus' value for each postcode_area_m_y,
# for example for the g_median_price_per_bed row which contains values : 'AL10_01_2018, 117916.66' the list below will contain 'AL10_04_2018, 117916.66' 
# this means that for the postal area AL10 in April 2018, the median price per bedroom was 117916.66 three months ago.
list_of_months_plus_three=[]
for row in g_median_price_per_bed.itertuples():
   myMonth=int(str(row[1]).split('_')[1])
   myYear =int((str(row[1])).split('_')[2])
   threeMnthsPlus=setThreeMonthsPlus(row[1],myMonth,myYear)
   new_row=[]
   new_row.append(threeMnthsPlus)
   new_row.append(row[2])
   list_of_months_plus_three.append(new_row)


In [18]:
#data from the list above is placed in the g_median_price_per_bed dataframe (it overwrites the data which was there before)
g_median_price_per_bed = pd.DataFrame(list_of_months_plus_three)
g_median_price_per_bed.columns =[ 'postcode_area_m_y','median_ppb_3mnths_ago', ]
g_median_price_per_bed

,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_04_2018,117916.666667
1,AL10_05_2018,121666.666667
2,AL10_06_2018,118666.666667
3,AL10_07_2018,131666.666667
4,AL10_08_2018,122500.000000
...,...,...
23890,YO8_09_2018,60000.000000
23891,YO8_10_2018,66000.000000
23892,YO8_11_2018,62083.333333
23893,YO8_12_2018,65500.000000


In [19]:
#combining two dataframes: g_median_price_per_bed (historic [3m] median prices per bedrooms  for postal areas starting from 04_2018) and df_add_assumption (as we decided about the assumption for 01_2018, 02_2018, 03_2018) 
g_median_price_per_bed_3m_ago = pd.concat([df_add_assumption, g_median_price_per_bed], axis=0)
g_median_price_per_bed_3m_ago

,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_01_2018,117916.666667
1,AL10_02_2018,121666.666667
2,AL10_03_2018,118666.666667
12,AL1_01_2018,189083.333333
13,AL1_02_2018,183202.500000
...,...,...
23890,YO8_09_2018,60000.000000
23891,YO8_10_2018,66000.000000
23892,YO8_11_2018,62083.333333
23893,YO8_12_2018,65500.000000


In [25]:
  #downloading csv file with g_median_price_per_bed_3m_ago data, so the median priced per bedroom data could be kept in a db
  from google.colab import files
  m_from='01_2018'
  m_to='01_2019'
  my_file='median_price_per_bedroom_three_months_ago_period_{}_{}.csv'.format(m_from, m_to)
  with open(my_file, 'w') as f:
    g_median_price_per_bed_3m_ago.to_csv(my_file, index=False)
  files.download(my_file)  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
#Now mergin the sales data with g_median_price_per_bed_3m_ago, so each property sale transaction row contains 'median_ppd_3_mnths_ago'
data_prep = pd.merge(data_prep, g_median_price_per_bed_3m_ago, how="left", on="postcode_area_m_y")
data_prep

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom,median_ppb_3mnths_ago
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,M30_07_2018,100000.000000,60250.000000
1,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,NR3_04_2018,128000.000000,73416.666667
2,79A74E21-DB3C-1289-E053-6B04A8C01627,1,33000.0,LIVERPOOL,LIVERPOOL,L,F,21/09/2018,L5 4TD,L5_09_2018,33000.000000,25562.500000
3,79A74E21-1F11-1289-E053-6B04A8C01627,1,405000.0,LONDON,HAMMERSMITH AND FULHAM,L,F,27/09/2018,W14 9BW,W14_09_2018,405000.000000,342528.000000
4,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH,CB9_03_2018,156500.000000,91666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
541269,773788C2-69BA-2CE4-E053-6C04A8C05E57,16,1445000.0,AXMINSTER,EAST DEVON,F,D,25/07/2018,EX13 8AA,EX13_07_2018,90312.500000,65000.000000
541270,773788C2-A90F-2CE4-E053-6C04A8C05E57,17,1011992.0,LYNDHURST,NEW FOREST,F,D,10/08/2018,SO43 7BT,SO43_08_2018,59528.941176,171666.666667
541271,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA,NP4_01_2018,8235.294118,36008.333333
541272,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA,WV2_01_2018,3478.260870,36666.666667


In [35]:
#for testing calculations for a specific area
df_co4_t=data_prep[(data_prep['postcode_area_m_y']=='CO4_11_2018')]
co4_median_price_per_bed_t = df_co4_t.groupby(['postcode_area_m_y'])['median_ppb_3mnths_ago'].median().reset_index()
df_co4_test=pd.concat([co4_median_price_per_bed, co4_median_price_per_bed_t ], axis=1)
df_co4_test

,postcode_area_m_y,price_per_bedroom,postcode_area_m_y,median_ppb_3mnths_ago
0,CO4_08_2018,94500.0,CO4_11_2018,94500.0


In [21]:
#some of the postal areas would not have sale transactions three months ago.
#such transactions will have Nan median_ppb_3mnths_ago
data_test = data_prep[(data_prep['median_ppb_3mnths_ago'].isnull())]
data_test

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom,median_ppb_3mnths_ago
234,75050A84-D2F3-9A88-E053-6B04A8C02390,1,97000.0,LISKEARD,CORNWALL,F,S,20/07/2018,PL14 3QN,PL14_07_2018,97000.000000,NaN
248,773788C2-5C25-2CE4-E053-6C04A8C05E57,1,134500.0,BOSCASTLE,CORNWALL,F,T,16/08/2018,PL35 0AL,PL35_08_2018,134500.000000,NaN
551,7E86B6FB-1C9D-458C-E053-6B04A8C0C84C,1,450000.0,LONDON,ISLINGTON,L,F,17/08/2018,WC1X 9AA,WC1X_08_2018,450000.000000,NaN
558,7C2D0700-652F-4963-E053-6B04A8C07B97,1,129000.0,CHATHILL,NORTHUMBERLAND,F,T,15/10/2018,NE67 5BP,NE67_10_2018,129000.000000,NaN
613,726BF13B-4FD8-0A46-E053-6C04A8C01D0D,1,112500.0,SHEFFIELD,SHEFFIELD,L,T,01/05/2018,S14 1BZ,S14_05_2018,112500.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
541051,726BF13A-8A1F-0A46-E053-6C04A8C01D0D,8,705000.0,PLYMOUTH,CITY OF PLYMOUTH,F,D,18/07/2018,PL3 5NB,PL3_07_2018,88125.000000,NaN
541088,726BF13A-8D76-0A46-E053-6C04A8C01D0D,8,310000.0,LIFTON,WEST DEVON,F,D,29/06/2018,PL16 0AN,PL16_06_2018,38750.000000,NaN
541106,75050A85-D125-9A88-E053-6B04A8C02390,8,857500.0,CORWEN,DENBIGHSHIRE,F,D,19/07/2018,LL21 9EG,LL21_07_2018,107187.500000,NaN
541107,7011B109-2063-8ED6-E053-6B04A8C075C1,8,215000.0,PORT ISAAC,CORNWALL,F,T,05/06/2018,PL29 3SR,PL29_06_2018,26875.000000,NaN


In [26]:
#Checking how many postal areas do not have median_ppb_3mnths_ago
g_test = data_test.groupby(['postcode_area_m_y'])['unique_id'].count().reset_index()
g_test.rename(columns = {'unique_id':'num of prop transactions'}, inplace = True)
g_test


,postcode_area_m_y,num of prop transactions
0,B19_06_2018,1
1,B3_09_2018,1
2,B5_11_2018,1
3,B7_07_2018,1
4,B7_12_2018,1
...,...,...
478,WC2N_04_2018,1
479,WV15_04_2018,2
480,WV9_06_2018,10
481,YO51_06_2018,5


In [28]:
  #downloading csv file with postcode_area_m_y which do not have the 3m-ago median prices per bedroom. 
  #these postcode_area_m_y to be put on a db, so an alternative value could be assigned later to these postcode_areas_m_y
  m_from='01_2018'
  m_to='01_2019'
  my_file='postcode_areas_m_y_without_prop_transact_three_months_ago_{}_{}.csv'.format(m_from, m_to)
  with open(my_file, 'w') as f:
    g_test.to_csv(my_file, index=False)
  files.download(my_file)  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>